In [16]:
def loadFile(dataPath,labelPath):
    dataList = []
    labelList= []
    #打开文件：以二进制读模式、utf-8格式的编码方式打开
    frData = open(dataPath)
    frLabel = open(labelPath)
    dataRecord = frData.read()
    labelRecord = frLabel.read()
    # print(record)
    frData.close()
    frLabel.close()
    #按照行转换为一维表即包含各行作为元素的列表，分隔符有'\r', '\r\n', \n'
    dataRecordList = dataRecord.splitlines()
    labelRecordList=labelRecord.splitlines()
    #逐行遍历：行内字段按'\t'分隔符分隔，转换为列表
    for line in dataRecordList:
        if line.strip():
            a=line.split('\t')
            a.pop()
            a=list(map(float, a))
            dataList.append(a)
    for line in labelRecordList:
            if line.strip():
                a=line.split()
                # a.pop()
                a=list(map(float, a))
                if a[0]!=0:
                    labelList.append(1)
                else:
                    labelList.append(0)
            # dataList.append(list(line.split("\t")))
    #返回转换后的矩阵
    x = np.array(dataList)
    y = np.array(labelList)
    return x,y
    # return dataList

In [17]:
import numpy as np
dataPath ="./data.txt"
labelPath = "./label.txt"
tx,ty=loadFile(dataPath,labelPath)


In [18]:
print(len(tx[0]))
print(len(tx),len(ty))
print(ty[:100])

300
88851 88851
[0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1
 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0]


In [28]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split


inputs = keras.Input(shape=(300,))
x = layers.Reshape((300,1))(inputs)
# Embed each integer in a 128-dimensional vector
# x = layers.Embedding(1000, 30)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(128))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

callbacks = [
    keras.callbacks.EarlyStopping(
        # 不再提升的关注指标
        monitor='val_loss',
        # 不再提升的阈值
        min_delta=1e-3,
        # 不再提升的轮次
        patience=2,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
    # 模型路径
    filepath='./BLSTM/model_{epoch}.h5',
    # 是否保存最佳
    save_best_only=True,
    # 监控指标
    monitor='val_accuracy',
    # 进度条类型
    verbose=1
    )
]

model.compile("adam",
              "binary_crossentropy",
              metrics=["accuracy",
                       "Precision",
                       "Recall",
                       "TruePositives",
                       "TrueNegatives",
                       "FalsePositives",
                       "FalseNegatives"])



Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 300, 1)            0         
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 300, 256)          133120    
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 256)               394240    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 527,617
Trainable params: 527,617
Non-trainable params: 0
_________________________________________________________________


In [ ]:

train_x = tx[:60000]
train_y = ty[:60000]
print(len(train_x),len(train_x[0]))


model.fit(train_x, train_y, epochs=10, batch_size=64,validation_split=0.2,callbacks=callbacks)
model.save('./first_model.h5')

60000 300
Epoch 1/10
 25/750 [>.............................] - ETA: 1:00:27 - loss: 0.6093 - accuracy: 0.6687 - precision: 0.6738 - recall: 0.9282 - true_positives: 944.0000 - true_negatives: 126.0000 - false_positives: 457.0000 - false_negatives: 73.0000

In [22]:
model.save('./first_model.h5')

In [30]:
model = keras.models.load_model('./first_model.h5')
# model1.summary()
model.fit(train_x, train_y, epochs=4, batch_size=100,validation_split=0.2)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 128)          128000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 300, 128)          98816     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 325,761
Trainable params: 325,761
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
80/80 [==============================] - 224s 3s/step - loss: 0.1677 - accuracy: 0.9255 - val_loss: 0.4480 - 

In [86]:
tt=int(np.random.randint(30000,80000))
ttx=tt+1000
print(tt,ttx)

59966 60966


In [26]:
tt=int(np.random.randint(30000,80000))
ttx=tt+1000
# print(tt,ttx)
test_x = tx[-1000:]
test_y = ty[-1000:]
model = keras.models.load_model('./first_model.h5')
results = model.evaluate(test_x, test_y, batch_size=64)
# print("test loss, test acc:", results)

# predictions = model.predict(test_x)
# list =[]
# # print("prediction:", predictions[0][0])
# for i in range(len(test_x)):
#     # print("prediction:", predictions[i][0])
#     if predictions[i][0] >= 0.5:
#         list.append(1)
#     else:
#         list.append(0)
# x=0
# for i in range(len(test_x)):
#     if list[i]==test_y[i]:
#         x+=1
#
# # print(test_y[:100])
# print(x/len(test_x))

16/16 [==============================] - 5s 162ms/step - loss: 0.4815 - accuracy: 0.8270 - precision: 0.8730 - recall: 0.8339 - true_positives: 502.0000 - true_negatives: 325.0000 - false_positives: 73.0000 - false_negatives: 100.0000
